In [17]:
import argparse
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.multiprocessing as mp
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

In [3]:
data = "data"
batch_size = 256
workers = 4
valdir = os.path.join(data, 'val')

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

val_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(valdir, transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ])),
    batch_size=batch_size, shuffle=False,
    num_workers=workers, pin_memory=True)

In [4]:
model = torch.hub.load('facebookresearch/WSL-Images', 'resnext101_32x16d_wsl')

Using cache found in /home/liqi/.cache/torch/hub/facebookresearch_WSL-Images_master


In [5]:
device = "cuda:7"
model = model.cuda(device)

In [6]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [12]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

def validate(val_loader, model, criterion, print_freq):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):

            images = images.cuda(device)
            target = target.cuda(device)
            
            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))

    return top1.avg

In [8]:
criterion = nn.CrossEntropyLoss()

In [13]:
acc1 = validate(val_loader, model, criterion, 10)

Test: [ 0/40]	Time  3.384 ( 3.384)	Loss 1.2066e+01 (1.2066e+01)	Acc@1   0.00 (  0.00)	Acc@5   1.17 (  1.17)
Test: [10/40]	Time  1.858 ( 2.003)	Loss 1.4034e+01 (1.3526e+01)	Acc@1   0.00 (  0.04)	Acc@5   0.00 (  1.03)
Test: [20/40]	Time  1.856 ( 1.933)	Loss 1.0802e+01 (1.3274e+01)	Acc@1   0.00 (  0.02)	Acc@5   0.39 (  0.61)
Test: [30/40]	Time  1.872 ( 1.909)	Loss 1.3873e+01 (1.3898e+01)	Acc@1   0.00 (  0.03)	Acc@5   0.78 (  0.55)
 * Acc@1 0.020 Acc@5 0.450


In [14]:
import pandas as pd

In [32]:
pred = pd.read_csv('output.csv')

In [33]:
pred

,Id,Category
0,0.jpg,916
1,1.jpg,107
2,2.jpg,611
3,3.jpg,510
4,4.jpg,611
5,5.jpg,741
6,6.jpg,554
7,7.jpg,611
8,8.jpg,904
9,9.jpg,611


In [34]:
myoutput = pd.read_csv('pred.txt')

In [38]:
pred['Category'] = myoutput['pred']

In [41]:
pred.to_csv('results.csv', index=0)